In [70]:
import pandas as pd
from datetime import datetime


In [71]:
import re

global x, y
df = pd.read_csv('../static/data.csv', header=None, encoding='utf-8-sig')
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,上海鸿帆网络科技有限公司,Python开发工程师,本科,民营,少于50人,上海·徐汇区,16000,"2年, 本科, python, fastapi, django, redis, docker...",2年,全职,2023-08-28,计算机服务
1,网思科技股份有限公司,Python工程师,本科,民营,500-1000人,昆明·西山区,8000,"1年, 本科, rpa, python, 周末双休, 带薪年假, 大牛带队, 朝九晚六",1年,全职,2023-08-30,计算机服务
2,深圳市悠享网络科技有限公司,Python工程师,大专,创业公司,50-150人,深圳·宝安区,13000,"3-4年, 大专, 数据库, 后台管理系统, ccs, python, js, html, ...",3-4年,全职,2023-08-29,网络游戏
3,广东众图科技有限公司,Python开发高级工程师,大专,民营,150-500人,佛山·南海区,12000,"5-7年, 大专, 五险一金, 周末双休, 带薪年假, 专业培训, 绩效奖金, 团队聚餐, ...",5-7年,全职,2023-08-28,互联网/电子商务
4,宁波草船科技有限公司,Python工程师,本科,民营,少于50人,宁波·鄞州区,13500,"1年, 本科, 计算机软件, 机器学习, 算法, 数据库, 多线程, go, 云服务, Py...",1年,全职,2023-08-30,互联网/电子商务
...,...,...,...,...,...,...,...,...,...,...,...,...
2247,首汽租赁有限责任公司,首汽租车西宁车辆资产管理员,大专,国企,1000-5000人,西宁,4500,"2年, 大专, 五险一金",2年,全职,2023-08-04,交通/运输/物流
2248,东莞市奥利汽车销售服务有限公司,车辆管理员,无需学历,民营,150-500人,东莞,6500,"2年, 驾驶, 带薪年假, 节日福利, 绩效奖金, 餐饮补贴, 包吃, 五险, 定期体检",2年,全职,2023-08-02,汽车
2249,皖江金融租赁股份有限公司,车辆金融事业部业务岗,本科,国企,150-500人,合肥·滨湖新区,22500,"3-4年, 本科, 市场营销, 金融, 车辆工程, 项目评估论证, 汽车金融公司, 金融租赁...",3-4年,全职,2023-08-08,金融/投资/证券
2250,AKKA技术有限公司,车辆内饰采购,本科,外资,150-500人,上海·浦东新区,11500,"无需经验, 本科, 供应商谈判, 供应商关系, 全球采购, 免费班车, 员工旅游, 专业培训...",无需经验,全职,2023-08-15,专业服务


In [72]:
temp = pd.to_datetime(df[10])
df[10] = temp.apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))
df = df.replace('[(（].*[）)]', '', regex=True)
df.drop_duplicates(keep='first', inplace=True)# 删除重复值
df.reset_index(drop=True, inplace=True)
for i in range(0, df.shape[0]):  # 0-1349 数据条数a
    try:
        s = df.loc[[i], [6]].values.tolist()[0][0]
        if s[-3:-1] == '下/':
            df.loc[[i], [6]] = int(eval(s[0])* 10000 / 12)
            continue
        if s[-1] == '天':
            if s[-3]=='千':
                df.loc[[i], [6]] = int(eval(s[0:-3]) *1000* 22)
                continue
            elif s[-3]=='万':
                df.loc[[i], [6]] = int(eval(s[0:-3]) *10000* 22)
                continue
            else:
                df.loc[[i], [6]] = int(eval(s[0:-3]) * 22)
                continue
        if str(s) == 'nan':
            continue
        if str(s)[-1] == '下':
            w = re.search('([0-9]*[万千])', s)
            if str(w[0])[-1] == '万':
                r = int(str(w[0])[0]) * 10000
            else:
                r = int(str(w[0])[0]) * 1000
            value = round(r / 2)
            df.loc[[i], [6]] = value
        else:
            a = re.search('(.*)-([0-9.]*[\u4e00-\u9fa5])', str(s))[1]
            b = re.search('(.*)-([0-9.]*[\u4e00-\u9fa5])', str(s))[2]
            if b[-1] == '千':
                x = eval(a[0:]) * 1000
                y = eval(b[0:-1]) * 1000
            elif a[-1] == '千':
                x = eval(a[0:-1]) * 1000
                y = eval(b[0:-1]) * 10000
            elif b[-1] == '万':
                x = eval(a[0:]) * 10000
                y = eval(b[0:-1]) * 10000
            elif b[-1] == '年':
                a1 = re.search('(.*)-(.*)\/年', str(s))[1]  # 处理年薪
                b1 = re.search('(.*)-(.*)\/年', str(s))[2]
                x1 = eval(a1[0:]) * 10000
                y1 = eval(b1[0:-1]) * 10000
                value1 = round((x1 + y1) / (2 * 12), -1)
                df.loc[[i], [6]] = value1
            elif b[-1] == '薪':
                m = re.search('(.*)-([0-9.]*[\u4e00-\u9fa5])', a)[1]
                n = re.search('(.*)-([0-9.]*[\u4e00-\u9fa5])', a)[2]
                if n[-1] == '千':
                    x = eval(a[0:]) * 1000
                    y = eval(b[0:-1]) * 1000
                elif m[-1] == '千':
                    x = eval(a[0:-1]) * 1000
                    y = eval(b[0:-1]) * 10000
                elif n[-1] == '万':
                    x = eval(a[0:]) * 10000
                    y = eval(b[0:-1]) * 10000
                else:
                    print(s)
                    continue
            else:
                print(s)
                continue
            value = round((x + y) / 2)
            df.loc[[i], [6]] = value
    except:
        continue

In [73]:
df[6].fillna(df[6].mean(), inplace=True)
df[2].fillna('无需学历', inplace=True)
df[3].fillna('其他', inplace=True)
df[4].fillna('暂无', inplace=True)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,上海鸿帆网络科技有限公司,Python开发工程师,本科,民营,少于50人,上海·徐汇区,16000,"2年, 本科, python, fastapi, django, redis, docker...",2年,全职,2023-08-28,计算机服务
1,网思科技股份有限公司,Python工程师,本科,民营,500-1000人,昆明·西山区,8000,"1年, 本科, rpa, python, 周末双休, 带薪年假, 大牛带队, 朝九晚六",1年,全职,2023-08-30,计算机服务
2,深圳市悠享网络科技有限公司,Python工程师,大专,创业公司,50-150人,深圳·宝安区,13000,"3-4年, 大专, 数据库, 后台管理系统, ccs, python, js, html, ...",3-4年,全职,2023-08-29,网络游戏
3,广东众图科技有限公司,Python开发高级工程师,大专,民营,150-500人,佛山·南海区,12000,"5-7年, 大专, 五险一金, 周末双休, 带薪年假, 专业培训, 绩效奖金, 团队聚餐, ...",5-7年,全职,2023-08-28,互联网/电子商务
4,宁波草船科技有限公司,Python工程师,本科,民营,少于50人,宁波·鄞州区,13500,"1年, 本科, 计算机软件, 机器学习, 算法, 数据库, 多线程, go, 云服务, Py...",1年,全职,2023-08-30,互联网/电子商务
...,...,...,...,...,...,...,...,...,...,...,...,...
2247,首汽租赁有限责任公司,首汽租车西宁车辆资产管理员,大专,国企,1000-5000人,西宁,4500,"2年, 大专, 五险一金",2年,全职,2023-08-04,交通/运输/物流
2248,东莞市奥利汽车销售服务有限公司,车辆管理员,无需学历,民营,150-500人,东莞,6500,"2年, 驾驶, 带薪年假, 节日福利, 绩效奖金, 餐饮补贴, 包吃, 五险, 定期体检",2年,全职,2023-08-02,汽车
2249,皖江金融租赁股份有限公司,车辆金融事业部业务岗,本科,国企,150-500人,合肥·滨湖新区,22500,"3-4年, 本科, 市场营销, 金融, 车辆工程, 项目评估论证, 汽车金融公司, 金融租赁...",3-4年,全职,2023-08-08,金融/投资/证券
2250,AKKA技术有限公司,车辆内饰采购,本科,外资,150-500人,上海·浦东新区,11500,"无需经验, 本科, 供应商谈判, 供应商关系, 全球采购, 免费班车, 员工旅游, 专业培训...",无需经验,全职,2023-08-15,专业服务


In [74]:
df = df.astype({6: 'int'})
df1 = df[df[4].notnull()]
df1.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
for i in range(df.shape[0]):
    df[7][i] = df[7][i].replace("'", '')
df.to_csv('../static/data.csv', index=False, header=False, encoding='utf-8-sig')
print('数据处理完毕！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！')

C:\Users\Polo\AppData\Local\Temp\ipykernel_18684\288472921.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[7][i] = df[7][i].replace("'", '')


数据处理完毕！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！


In [77]:
pd.to_numeric(df['column1'], errors='coerce').mean()

,0,1,2,3,4,5,6,7,8,9,10,11


In [76]:
df[df[5].isnull()]

,0,1,2,3,4,5,6,7,8,9,10,11


In [53]:
int(eval(dd[0:-3]) *1000* 22)

33000

In [154]:
import configparser

cfp = configparser.RawConfigParser()
cfp.read('../config.ini', encoding='utf-8')
t=bool(int(cfp.get('setting', 'reget_data')))

In [155]:
t

True

In [151]:
if t :
    print('yes')

['0']

In [148]:
string_value = '0'
boolean_value = bool(string_value)
print(boolean_value)  # 输出: True


True


In [140]:
string_value = "abc"
boolean_value = bool(string_value.upper())  # 转为大写后再进行转换
print(boolean_value)  # 输出: False


True


In [141]:
print(bool("abc"))

True
